<a href="https://colab.research.google.com/github/JorgeJaramilo060892/Testrepo/blob/main/Launch_Sites_Locations_Analysis_with_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Launch Sites Locations Analysis

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. we will be performing more interactive visual analytics using `Folium`.

## Objectives




*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the tasks, we should be able to find some geographical patterns about launch sites.

Let's first import required Python packages:

In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

###Task 1: Mark all launch sites on a map

In [5]:
from urllib import request

URL =  'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = request.urlopen(URL)
spacex_df= pd.read_csv(URL)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [6]:
# Let's select the relevant sub-columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give us any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

### NASA


In [7]:
# Start Location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Let's create a blue circle at NASA Johnson Space Center's coordinate with a popu label showing its name
# We could use folium.cicrcle to add a highlighted circe area with a text label on a specific coordinate
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Let's create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

As we can see, we should find a small orange circle near the city of Houston, we can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [10]:
# Initial the map
# For each launch site, let's add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Create the DataFrame with the data
launch_sites_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745]
}

launch_sites_df = pd.DataFrame(launch_sites_data)

# Define the center of the map as the coordinates of the first launch site
map_center = [launch_sites_df['Lat'].iloc[0], launch_sites_df['Long'].iloc[0]]
site_map = folium.Map(location=map_center, zoom_start=5)

# Add circles and markers
for index, site in launch_sites_df.iterrows():
    coordinate = [site['Lat'], site['Long']]
    label = site['Launch Site']

    # Add a circle for each launch site
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(label))
    site_map.add_child(circle)

    # Add a marker with a label for each launch site
    marker = folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label,
        ),
    )
    site_map.add_child(marker)

# Show the map
site_map


Now, we can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

As we can see in the map that we have just generated with Folium:


1.- No launch site is located near the equator.

2.- All launch sites are located near the coast, this is probably in case of an emergency.

### Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [24]:

# Let's apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
launch_sites_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [25]:
# For each launch result in the DataFrame spacex_df, add a folium.Marker to marker_cluster
for index, row in spacex_df.iterrows():
    # Get launch information
    launch_site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    marker_color = row['marker_color']

    # Create the marker and add it to the cluster
    marker_cluster.add_child(folium.Marker(location=[lat, long], popup=f'{launch_site}: {row["class"]}', icon=folium.Icon(color=marker_color)))

# Create a map
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=10)

# Add marker_cluster to the site_map
site_map.add_child(marker_cluster)

# Show the map
site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


### TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [26]:
# Let's add mouse position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright',
    separator = 'Long: ',
    empty_string= 'NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat: ',
    lat_formatter=formatter,
    lng_formatter=formatter,

)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [27]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, long2):
  # approximate radius of earth in km
  R = 6373.0

  lat1 = radians(lat1)
  lon1 = radians(lon1)
  lat2 = radians(lat2)
  long2 = radians(long2)

  dlon = long2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat /2)**2 + cos(lat1) * cos(lat2) *sin(dlon /2)**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))

  distance = R * c
  return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [28]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

coastline_lat = 28.56367
coastline_lon = -80.57163

# Create a new map
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=10)

# Create an object MarkerCluster
marker_cluster = MarkerCluster()

# For each launch result in the DataFrame spacex_df, add a folium.Marker to marker_cluster
for index, row in spacex_df.iterrows():
    # Get launch information
    launch_site = row['Launch Site']
    lat = row['Lat']
    lon = row['Long']
    marker_color = row['marker_color']

    # Create the marker and add it to the cluster
    marker = folium.Marker(location=[lat, lon], popup=f'{launch_site}: {row["class"]}', icon=folium.Icon(color=marker_color))
    marker_cluster.add_child(marker)

    # Calculate the distance to the closest coastline
    distance_coastline = calculate_distance(lat, lon, coastline_lat, coastline_lon)

    # Add a circle marker to represent the closest coastline point
    folium.CircleMarker(location=[coastline_lat, coastline_lon], radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(site_map)

    # Create a marker for the distance and add it to the map
    distance_marker = folium.Marker(
        location=[coastline_lat, coastline_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
        )
    )
    distance_marker.add_to(site_map)

    # Print the distance to the closest coastline
    print(f"Distance from {launch_site} to the closest coastline: {distance_coastline} km")

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Show the map
site_map

Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the closest coastline: 0.5797581813109574 km
Distance from CCAFS LC-40 to the close

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [29]:
# Create a folium.PolyLine object using the coastline coordinates and launch site coordinate
polyline_coordinates = [[lat, lon], [coastline_lat, coastline_lon]]
polyline = folium.PolyLine(locations=polyline_coordinates, weight=1)

# Add the PolyLine to the map
site_map.add_child(polyline)

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [30]:
# Example coordinates of the closest city, railway, highway, etc.
closest_city_lat = 28.56300
closest_city_lon = -80.57000

# Calculate the distance to the closest city
distance_city = calculate_distance(lat, lon, closest_city_lat, closest_city_lon)

# Create a marker for the distance to the closest city and add it to the map
city_marker = folium.Marker(
    location=[closest_city_lat, closest_city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#3498db;"><b>{distance_city:.2f} KM to City</b></div>'
    )
)
city_marker.add_to(site_map)

# Draw a PolyLine between the launch site and the closest city
city_polyline_coordinates = [[lat, lon], [closest_city_lat, closest_city_lon]]
city_polyline = folium.PolyLine(locations=city_polyline_coordinates, weight=1)
site_map.add_child(city_polyline)


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

As we can see the launch sites:
- 1.- The launch sites are indeed close to the railway lines
- 2.-The launch sites are indeed near some highways
- 3.- Launch sites effectively close to the coastline
- 4.- The launch sites are approximately far from the cities.